In [ ]:
!pip install -q --upgrade diffusers transformers accelerate peft streamlit

In [ ]:
%%writefile app.py

import streamlit as st
from diffusers import DiffusionPipeline, AutoPipelineForText2Image
import torch
import time

# Function to perform inference and display the result
def perform_inference(prompt, num_inference_steps, guidance_scale, use_gpu, manual_seed):

    # Initialize the Diffusion Pipeline
    if use_gpu:
        # Run on GPU
        pipeline = AutoPipelineForText2Image.from_pretrained("stabilityai/sdxl-turbo", torch_dtype=torch.float16, variant="fp16")
        pipeline.to("cuda")
    else:
        # Run on CPU
        pipeline = DiffusionPipeline.from_pretrained("n1i/sdxl-ssd-1b-lcm")

    # Measure latency
    start_time = time.time()

    generator = torch.Generator(device=pipeline.device).manual_seed(manual_seed)
    # Perform inference
    result = pipeline(
        prompt=prompt,
        num_inference_steps=num_inference_steps,
        guidance_scale=guidance_scale
    ).images[0]

    end_time = time.time()
    latency_seconds = end_time - start_time
    latency_minutes = int(latency_seconds // 60)
    latency_seconds = int(latency_seconds % 60)

    # Display the image
    st.image(result, caption='Generated Image')

    # Print the latency
    st.write("Latency:", latency_minutes, "minutes and", latency_seconds, "seconds")

def main():
    # Set page title
    st.title("SDXL SSD 1B LCM")

    # Get prompt input from user
    prompt = st.text_input("Enter your prompt:", "A cinematic shot of a baby raccoon wearing an intricate Italian priest robe, 8k.")

    # Create a sidebar for options
    st.sidebar.title("Options")
    num_inference_steps = st.sidebar.slider("Number of Inference Steps", min_value=1, max_value=10, value=1)
    guidance_scale = st.sidebar.slider("Guidance Scale", min_value=0.0, max_value=1.0, value=0.0)

    # Add slider to set manual_seed value
    manual_seed = st.sidebar.slider("Manual Seed", min_value=0, max_value=9999, value=1337)

    # Add radio button to choose GPU or CPU with CPU as default
    use_gpu = st.sidebar.radio("Select device:", ("CPU", "GPU"), index=0)

    # Perform inference when the button is clicked
    if st.button("Generate Image"):
        perform_inference(prompt, num_inference_steps, guidance_scale, use_gpu == "GPU", manual_seed)

if __name__ == "__main__":
    main()


In [ ]:
!npm install localtunnel

In [ ]:
import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

In [ ]:
!streamlit run app.py &>/content/logs.txt & npx localtunnel --port 8501